## Proyecto Final Special Topics <br>
Diego Rafael de Lemos Burgaña <br>
Universidad de Navarra, diciembre 2022

### Introduccion <br>
La compra de un nuevo portatil ya sea para uso personal, de trabajo o para gaming es un proceso tedioso debido a la cantidad de paginas, promociones y opciones que son cada vez mas grandes. <br> 
Dentro de España existen varias tiendas online que permiten una mayor flexibilidad y facilidad al realizar la compra pero siguen siendo muchas opciones, por eso para este proyecto he decidido scrapear la informacion y los precios de los ordenadores mas vendidos en las tiendas mas usadas dentro del país para poder compararlos y llegar a una decision de una manera mas rapida. Dentro de los datos scrapeados vamos a poder filtrar las opciones para llegar al portatil con mejor precio para nuestros requisitos.<br>
<br>
He decidido utilizar las siguientes tiendas:<br>
<li>MediaMarkt: Share de visitas = 21,43% <br>
<li>Pc Componentes: Share de visitas = 14,78% <br>
<li>Worten: Share de visitas = 5,63%<br>
<li>PhoneHouse = Share de visitas 4,58%<br>
<li>Back Market = Share de visitas 1,56%<br>
<br>
Con estas opciones estariamos scrapeando alrededor del 50% de las paginas mas visitadas para la compra de ordenadores portatiles dentro de España.<br>
<br>
Estos datos extraidos de las paginas luego van ser guardados y van a poder utilizarse para un modelo de Machine Learning


## WebScraping de los datos

### MediaMarkt

Importamos los paquetes necesarios para el proyecto

In [202]:
from tabulate import tabulate
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
from urllib.error import HTTPError
from urllib.error import URLError
import re
import requests
import pandas as pd
import random 

Establecemos los user agents y los headers que nos van a permitir acceder a la mayoria de las paginas sin problemas por bloqueos a bots de WebScraping

In [203]:
user_agents = [ 
	'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36', 
	'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36', 
	'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36', 
	'Mozilla/5.0 (iPhone; CPU iPhone OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148', 
	'Mozilla/5.0 (Linux; Android 11; SM-G960U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Mobile Safari/537.36',
	'Accept' 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'

] 
user_agent = random.choice(user_agents) 
headers = {'User-Agent': user_agent} 
response = requests.get('https://httpbin.org/headers', headers=headers)

Establecemos la url de MediaMarkt que queremos extraer e identificamos los valores que tiene que buscar, en este caso siendo los precios en descuento de los portatiles mas vendidos en la paginas.

In [206]:
url = "https://www.mediamarkt.es/es/category/port%C3%A1tiles-153.html?sort=salescount%2Bdesc"
request_site = Request(url, headers=headers)
webpage = urlopen(request_site).read()
bsObj = BeautifulSoup(webpage) 

MediaPriceDisc = bsObj.findAll("span", {'class':"BaseTypo-sc-1jga2g7-0 izkVco StyledInfoTypo-sc-1jga2g7-1 dNrVxe StyledPriceTypo-jah2p3-7 ddPMiu"})
media_laptop_price = []
for value in MediaPriceDisc:
    print(value.get_text())
    media_laptop_price.append(value.getText())

649.–
274.
999.–
229.–
529.–
999.–


Ahora extraemos el resto de los precios que no se encuentran en descuento

In [207]:
MediaPrice = bsObj.findAll("span", {'class':"BaseTypo-sc-1jga2g7-0 izkVco StyledInfoTypo-sc-1jga2g7-1 ikIrGc StyledPriceTypo-jah2p3-7 ddPMiu"})
for value in MediaPrice:
    print(value.get_text())
    media_laptop_price.append(value.getText())

369.–
449.–
599.–
366.–
347.–
449.–


Extraemos el nombre y las caracteristicas de cada una de los ordenadores portatiles dentro de la pagina

In [208]:
MediaLaptop = bsObj.findAll("p", {'class':"BaseTypo-sc-1jga2g7-0 izkVco StyledInfoTypo-sc-1jga2g7-1 doYUxh"})
media_laptop_name = []
for name in MediaLaptop:
    print(name.get_text())
    media_laptop_name.append(name.getText())

Portátil - ASUS F515JA-EJ2883W, 15.6" Full HD, Intel® Core™ i7-1065G7, 16GB RAM, 512GB SSD, Iris® Plus Graphics, Windows 11 Home
Portátil  - 82C3001WSP LENOVO, 15,6 ", Full-HD, Celeron, 8 GB, 256 GB, Intel UHD Graphics integrada, Windows 10 Home Gris
Apple MacBook Air (2020), 13.3" Retina, Chip M1 de Apple, 8 GB, 256 GB SSD, MacOS, Teclado Magic Keyboard Touch ID, Gris espacial
Portátil - HP 15S-EQ1040NS, 15.6" HD, AMD 3020e, 4GB RAM, 128GB SSD, AMD Radeon™, W11 S, Plata
Portátil - HP Laptop 15s-fq3002ns, 15.6" Full-HD, Intel® Celeron® N4500, 8GB RAM, 256GB SSD, Intel® UHD, Windows 11 Home
Portátil - ASUS Chromebook CX1400CNA-EK0179, 14" Full-HD, Intel® Celeron® N3350, 8 GB, 64 GB,  Chrome OS
Convertible 2 en 1 - Microsoft Surface Pro X, 13" QHD+, Microsoft SQ® 1, 8 GB RAM, 128 GB SSD, Adreno 685, W11
Portátil - ASUS F1500EA-BQ2362, 15.6" Full HD, Intel® Core™ i3-1115G4 , 8GB RAM, 256 GB SSD, UHD Graphics, Sin sistema operativo
Portátil - ASUS F1400EA-EK1838W, 14" Full HD, Intel® Core™

Realizamos una tabla en donde agrupamos todos los precios, junto con sus respectivos nombres y caracteristicas

In [209]:
dataMedia = [media_laptop_name, media_laptop_price]
MediaMarkt = pd.DataFrame(dataMedia).transpose()
MediaMarkt.rename(columns= {0:'Laptop Name', 1:'Price'})

,Laptop Name,Price
0,"Portátil - ASUS F515JA-EJ2883W, 15.6"" Full HD,...",649.–
1,"Portátil - 82C3001WSP LENOVO, 15,6 "", Full-HD...",274.
2,"Apple MacBook Air (2020), 13.3"" Retina, Chip M...",999.–
3,"Portátil - HP 15S-EQ1040NS, 15.6"" HD, AMD 3020...",229.–
4,"Portátil - HP Laptop 15s-fq3002ns, 15.6"" Full-...",529.–
5,"Portátil - ASUS Chromebook CX1400CNA-EK0179, 1...",999.–
6,"Convertible 2 en 1 - Microsoft Surface Pro X, ...",369.–
7,"Portátil - ASUS F1500EA-BQ2362, 15.6"" Full HD,...",449.–
8,"Portátil - ASUS F1400EA-EK1838W, 14"" Full HD, ...",599.–
9,"Portátil - Dynabook Satellite Pro C50-G-104, 1...",366.–


### PC Componentes

Para esta pagina primero tenemos que comprobar si se puede acceder a traves del bot para scrapear informacion, para esto usamos el siguiente codigo

In [210]:
try:
    html = urlopen("https://www.pccomponentes.com/portatiles/mas-vendidos")
except HTTPError as e:
    print('user tried to access a forbidden page!')
except URLError as e:
    print("The server could not be found!")
else:
    print("It Worked!")


user tried to access a forbidden page!


El resultado viene siendo que no se puede acceder debido a que no tenemos permiso por seguridad de la pagina.

### Worten

Realizamos el mismo proceso que con las otras paginas

In [211]:
url2 = "https://www.worten.es/productos/informatica/portatiles"
request_site2 = Request(url2, headers=headers)
webpage2 = urlopen(request_site2, timeout=30).read()
bsObj2 = BeautifulSoup(webpage2)

WortenCompPrices = bsObj2.findAll("span", {'class':"w-currentPrice iss-current-price"})
worten_laptop_price = []
for value in WortenCompPrices:
    print(value.get_text())
    worten_laptop_price.append(value.getText())

749,00€
529,00€
749,99€
699,00€
599,99€
369,00€
529,99€
649,00€
659,00€
999,00€
1279,00€
1499,00€
1399,00€
1302,38€
379,99€
1399,00€
649,99€
449,87€
699,00€
679,00€
219,00€
745,50€
999,00€
649,99€


Luego los nombres y caracteristicas de los ordenadores

In [212]:
WortenCompLaptop = bsObj2.findAll("h3", {'class':"w-product__title"})
worten_laptop_name = []
for name in WortenCompLaptop:
    print(name.get_text())
    worten_laptop_name.append(name.getText())

Portátil HP 15s-fq5049ns (15.6'' - Intel Core i7-1255U - RAM: 16 GB - 512 GB SSD - Intel Iris Xe Graphics)
Portátil ASUS F1500EA-EJ2382W (15.6 ''Intel Core i5-1135G7 - RAM: 8 GB  - 512 GB SSD - Intel UHD Graphics)
Portátil ACER Aspire 3 A315-58-56K2 (15.6'' - Intel Core i5-1135G7 - RAM: 12 GB - 512 GB SSD - Intel Iris Xe Graphics)
Portátil ACER Aspire 3 A315-58-79LA (15.6'' - Intel Core i7-1165G7 - RAM: 16 GB - 512 GB SSD - Intel Iris Xe Graphics)
Portátil HP 15s-eq2108ns (15.6'' - AMD Ryzen 5 5500U - RAM: 8 GB - 512 GB SSD PCIe - AMD Radeon Graphics)
Portátil HP 15S-FQ2168NS (15.6'' - Intel Core i3-1115G4 - RAM: 8 GB - 256 GB SSD - Intel UHD Graphics)
Portátil HP 15s-eq2111ns (15.6'' - AMD Ryzen 5 5500U - RAM: 8 GB - 512 GB SSD PCIe - AMD Radeon Graphics)
Portátil HP 15s-fq4000ns (15.6'' - Intel Core i5-1155G7 - RAM: 8 GB - 512 GB SSD - Intel Iris Xe Graphics)
Portátil ASUS VivoBook 15 OLED K513EA-L12891W (15.6'' - Intel Core i5-1135G7 - RAM: 12 GB - 512 GB SSD - Intel Iris Xe Graphic

Agrupamos todo dentro una misma tabla

In [213]:
dataWorten = [worten_laptop_name, worten_laptop_price]
Worten = pd.DataFrame(dataWorten).transpose()
Worten.rename(columns= {0:'Laptop Name', 1:'Price'})

,Laptop Name,Price
0,Portátil HP 15s-fq5049ns (15.6'' - Intel Core ...,"749,00€"
1,Portátil ASUS F1500EA-EJ2382W (15.6 ''Intel Co...,"529,00€"
2,Portátil ACER Aspire 3 A315-58-56K2 (15.6'' - ...,"749,99€"
3,Portátil ACER Aspire 3 A315-58-79LA (15.6'' - ...,"699,00€"
4,Portátil HP 15s-eq2108ns (15.6'' - AMD Ryzen 5...,"599,99€"
5,Portátil HP 15S-FQ2168NS (15.6'' - Intel Core ...,"369,00€"
6,Portátil HP 15s-eq2111ns (15.6'' - AMD Ryzen 5...,"529,99€"
7,Portátil HP 15s-fq4000ns (15.6'' - Intel Core ...,"649,00€"
8,Portátil ASUS VivoBook 15 OLED K513EA-L12891W ...,"659,00€"
9,Macbook Air APPLE MGN63Y/A Gris Espacial (13.3...,"999,00€"


### PhoneHouse

Importamos los datos de la pagina web de PhoneHouse<br>
Primero el precio de los portatiles

In [214]:
url4 = "https://www.phonehouse.es/informatica-y-tablets/ordenadores/portatiles.html"
request_site4 = Request(url4, headers=headers)
webpage4 = urlopen(request_site4, timeout=30).read()
bsObj4 = BeautifulSoup(webpage4)

PhoneHousePrices = bsObj4.findAll("span", {'class':"precio precio-2"})
ph_laptop_price = []
for value in PhoneHousePrices:
    print(value.get_text())
    ph_laptop_price.append(value.getText())

1549€
849€
379€
1995€
621,48€
2018,43€
1133,21€
479€
310€
369,95€
329€
450€
979€
2523,63€
809€
7323,54€
630€
14382,60€
749€
824,58€
967€
40,94€
23,54€
52,71€


Luego importamos los nombres y caracteristicas de los portatiles

In [217]:
PhoneHouseLaptop = bsObj4.findAll("h4", {'class':"marca-item"})
ph_laptop_name = []
for name in PhoneHouseLaptop:
    print(name.get_text())
    ph_laptop_name.append(name.getText())


					Samsung Galaxy Book2 Pro 15 i7 512GB+16GB RAM
				

					Samsung Galaxy Book2 15 INT i5 512GB+8GB RAM
				

					Lenovo ThinkPad Yoga 370 Negro Híbrido (2-en-1) 33,8 cm (13.3) 1920 x 1080 Pixeles Pantalla táctil 2,50 GHz 7ª generación de procesadores Intel® Core™ i5 i5-7200U
				

					Toshiba Portégé Z30t-C-133
				

					Toshiba Tecra X40-D-10G
				

					Microsoft Surface Laptop CI5 256GB 8GB SYST 13.5""
				

					Lenovo IdeaPad Y700 2.6GHz I7-6700HQ 15.6"" Negro
				

					Lenovo ThinkPad E580 Negro Portátil 39,6 cm (15.6) 1920 x 1080 Pixeles 1,60 GHz 8ª generación de procesadores Intel® Core™ i5 i5-8250U
				

					Apple MacBook Air Plata Portátil 29,5 cm (11.6) 1366 x 768 Pixeles 1,6 GHz 5ª generación de procesadores Intel® Core™ i5
				

					Lenovo ThinkPad X280 Negro Portátil 31,8 cm (12.5) 1920 x 1080 Pixeles 1,60 GHz 8ª generación de procesadores Intel® Core™ i5 i5-8250U
				

					Lenovo ThinkPad L470 Negro Portátil 35,6 cm (14) 1920 x 1080 Pixeles 2,50 GHz 7ª genera

Agrupamos los dos en un mismo dataframe

In [218]:
dataPhoneHouse = [ph_laptop_name, ph_laptop_price]
PhoneHouse = pd.DataFrame(dataPhoneHouse).transpose()
PhoneHouse.rename(columns= {0:'Laptop Name', 1:'Price'})

,Laptop Name,Price
0,\r\n\t\t\t\t\tSamsung Galaxy Book2 Pro 15 i7 5...,1549€
1,\r\n\t\t\t\t\tSamsung Galaxy Book2 15 INT i5 5...,849€
2,\r\n\t\t\t\t\tLenovo ThinkPad Yoga 370 Negro H...,379€
3,\r\n\t\t\t\t\tToshiba Portégé Z30t-C-133\r\n\t...,1995€
4,\r\n\t\t\t\t\tToshiba Tecra X40-D-10G\r\n\t\t\t\t,"621,48€"
5,\r\n\t\t\t\t\tMicrosoft Surface Laptop CI5 256...,"2018,43€"
6,\r\n\t\t\t\t\tLenovo IdeaPad Y700 2.6GHz I7-67...,"1133,21€"
7,\r\n\t\t\t\t\tLenovo ThinkPad E580 Negro Portá...,479€
8,\r\n\t\t\t\t\tApple MacBook Air Plata Portátil...,310€
9,\r\n\t\t\t\t\tLenovo ThinkPad X280 Negro Portá...,"369,95€"


### BackMarket

Importamos los datos de la pagina web de BackMarket

In [220]:
url3 = "https://www.backmarket.es/es-es/l/todos-los-portatiles-qwerty/3ea01fcb-7458-489b-84fa-c8c522b023bc"
request_site3 = Request(url3, headers=headers)
webpage3 = urlopen(request_site3, timeout=30).read()
bsObj3 = BeautifulSoup(webpage3)

BackMarketPrices = bsObj3.findAll("span", {'class':"body-2-bold text-black"})
bm_laptop_price = []
for value in BackMarketPrices:
    print(value.get_text())
    bm_laptop_price.append(value.getText())


                254,30 €
              

                399,00 €
              

                292,92 €
              

                349,00 €
              

                344,00 €
              

                199,00 €
              

                459,00 €
              

                199,00 €
              

                450,00 €
              

                258,36 €
              

                372,69 €
              

                229,00 €
              

                378,87 €
              

                249,00 €
              

                272,65 €
              

                220,00 €
              

                281,84 €
              

                289,45 €
              

                225,00 €
              

                425,00 €
              

                265,00 €
              

                325,00 €
              

                372,69 €
              

                225,00 €
              

               

Importamos los datos y caracteristicas de los ordenadores mas vendidos en BackMarket

In [221]:
BackMarketLaptop = bsObj3.findAll("p", {'class':"body-1-bold mb-1 text-black text-left flex-grow"})
bm_laptop_name = []
for name in BackMarketLaptop:
    print(name.get_text())
    bm_laptop_name.append(name.getText())


            HP EliteBook 840 G3 14” (2016)
          

            HP EliteBook 840 G3 14” (2020)
          

            Lenovo ThinkPad T470 14” (2017)
          

            Lenovo ThinkPad T470 14” (2017)
          

            HP EliteBook 840 G3 14” (2020)
          

            Dell Latitude E5450 14” (2015)
          

            HP EliteBook 840 G3 14” (2020)
          

            HP ProBook x360 11 G3 11,6” (Marzo 2019)
          

            HP EliteBook 840 G3 14” (2020)
          

            HP EliteBook 840 G3 14” (2016)
          

            Lenovo Thinkpad Yoga 260 12,5” (2015)
          

            Dell Latitude 3300 13,3” (2019)
          

            HP EliteBook 840 G5 14” (2021)
          

            Toshiba Satellite Pro C50-E-103 15,6” (2017)
          

            HP EliteBook 840 G2 14” (2015)
          

            Lenovo ThinkPad T450 14” (2015)
          

            Lenovo Thinkpad T460 14” (2016)
          

            HP EliteBook 840

Agrupamos los nombres y caracteristicas junto con los precios correspondientes

In [222]:
dataBM = [bm_laptop_name, bm_laptop_price]
BackMarket = pd.DataFrame(dataBM).transpose()
BackMarket.rename(columns= {0:'Laptop Name', 1:'Price'})

,Laptop Name,Price
0,\n HP EliteBook 840 G3 14” (2016)\n...,"\n 254,30 €\n"
1,\n HP EliteBook 840 G3 14” (2020)\n...,"\n 399,00 €\n"
2,\n Lenovo ThinkPad T470 14” (2017)\...,"\n 292,92 €\n"
3,\n Lenovo ThinkPad T470 14” (2017)\...,"\n 349,00 €\n"
4,\n HP EliteBook 840 G3 14” (2020)\n...,"\n 344,00 €\n"
5,\n Dell Latitude E5450 14” (2015)\n...,"\n 199,00 €\n"
6,\n HP EliteBook 840 G3 14” (2020)\n...,"\n 459,00 €\n"
7,"\n HP ProBook x360 11 G3 11,6” (Mar...","\n 199,00 €\n"
8,\n HP EliteBook 840 G3 14” (2020)\n...,"\n 450,00 €\n"
9,\n HP EliteBook 840 G3 14” (2016)\n...,"\n 258,36 €\n"


## Limpieza de datos <br>
Al extraer los datos de las distintas paginas podemos notar como unos se encuentra en distinto formato o mas desorganizados que otros. <br>
Ahora realizaremos la limpieza y estructura de estos datos para luego poder trabarlos de una manera mas ordenada por la evaluacion en Machine Learning

### Media Markt

In [223]:
MediaMarkt

,0,1
0,"Portátil - ASUS F515JA-EJ2883W, 15.6"" Full HD,...",649.–
1,"Portátil - 82C3001WSP LENOVO, 15,6 "", Full-HD...",274.
2,"Apple MacBook Air (2020), 13.3"" Retina, Chip M...",999.–
3,"Portátil - HP 15S-EQ1040NS, 15.6"" HD, AMD 3020...",229.–
4,"Portátil - HP Laptop 15s-fq3002ns, 15.6"" Full-...",529.–
5,"Portátil - ASUS Chromebook CX1400CNA-EK0179, 1...",999.–
6,"Convertible 2 en 1 - Microsoft Surface Pro X, ...",369.–
7,"Portátil - ASUS F1500EA-BQ2362, 15.6"" Full HD,...",449.–
8,"Portátil - ASUS F1400EA-EK1838W, 14"" Full HD, ...",599.–
9,"Portátil - Dynabook Satellite Pro C50-G-104, 1...",366.–


Reemplazamos el texto de "Portatil -" para dejar la data mejor estructurada

In [224]:
mediaName = [element.replace('Portátil - ', '') for element in media_laptop_name]
del mediaName[1]
mediaName = [x.split(',') for x in mediaName]

for i in range(len(mediaName)):
  del mediaName[i][6:8]
mediaName = pd.DataFrame(mediaName)
print(mediaName)


                                               0               1  \
0                            ASUS F515JA-EJ2883W   15.6" Full HD   
1                       Apple MacBook Air (2020)    13.3" Retina   
2                                HP 15S-EQ1040NS        15.6" HD   
3                         HP Laptop 15s-fq3002ns   15.6" Full-HD   
4               ASUS Chromebook CX1400CNA-EK0179     14" Full-HD   
5   Convertible 2 en 1 - Microsoft Surface Pro X        13" QHD+   
6                            ASUS F1500EA-BQ2362   15.6" Full HD   
7                           ASUS F1400EA-EK1838W     14" Full HD   
8               Dynabook Satellite Pro C50-G-104        15.6" HD   
9                       Apple MacBook Air (2020)    13.3" Retina   
10                         Acer Aspire 3 A315-56   15.6" Full HD   

                           2          3            4                     5  
0     Intel® Core™ i7-1065G7   16GB RAM    512GB SSD   Iris® Plus Graphics  
1           Chip M1 de Apple 

Limpiamos los precios del valor '.–'

In [225]:
mediaPrice = [element.replace('.–', '') for element in media_laptop_price]
del mediaPrice[1]
mediaPrice = pd.DataFrame(mediaPrice)
mediaPrice

,0
0,649
1,999
2,229
3,529
4,999
5,369
6,449
7,599
8,366
9,347


Los volvemos a agrupar dentro del dataframe

In [226]:
MediaMarkt = pd.merge(mediaName, mediaPrice, left_index=True, right_index=True)
MediaMarkt

,0_x,1,2,3,4,5,0_y
0,ASUS F515JA-EJ2883W,"15.6"" Full HD",Intel® Core™ i7-1065G7,16GB RAM,512GB SSD,Iris® Plus Graphics,649
1,Apple MacBook Air (2020),"13.3"" Retina",Chip M1 de Apple,8 GB,256 GB SSD,MacOS,999
2,HP 15S-EQ1040NS,"15.6"" HD",AMD 3020e,4GB RAM,128GB SSD,AMD Radeon™,229
3,HP Laptop 15s-fq3002ns,"15.6"" Full-HD",Intel® Celeron® N4500,8GB RAM,256GB SSD,Intel® UHD,529
4,ASUS Chromebook CX1400CNA-EK0179,"14"" Full-HD",Intel® Celeron® N3350,8 GB,64 GB,Chrome OS,999
5,Convertible 2 en 1 - Microsoft Surface Pro X,"13"" QHD+",Microsoft SQ® 1,8 GB RAM,128 GB SSD,Adreno 685,369
6,ASUS F1500EA-BQ2362,"15.6"" Full HD",Intel® Core™ i3-1115G4,8GB RAM,256 GB SSD,UHD Graphics,449
7,ASUS F1400EA-EK1838W,"14"" Full HD",Intel® Core™ i5-1135G7,8GB RAM,512GB SSD,Iris® Xe Graphics,599
8,Dynabook Satellite Pro C50-G-104,"15.6"" HD",Intel® Core™ i3-10110U,8GB RAM,256GB SSD,UHD,366
9,Apple MacBook Air (2020),"13.3"" Retina",Chip M1 de Apple,8 GB,256 GB SSD,MacOS,347


In [232]:
MediaMarkt.rename(columns = {'0_x':'Nombre Ordenador', 1:'Screen', 2: 'Processor', 3: 'RAM Memory', 4: 'Hard Drive', 5: 'Graphics Card', '0_y': 'Price (€)' }, inplace = True)
MediaMarkt

,Nombre Ordenador,Screen,Processor,RAM Memory,Hard Drive,Graphics Card,Price (€)
0,ASUS F515JA-EJ2883W,"15.6"" Full HD",Intel® Core™ i7-1065G7,16GB RAM,512GB SSD,Iris® Plus Graphics,649
1,Apple MacBook Air (2020),"13.3"" Retina",Chip M1 de Apple,8 GB,256 GB SSD,MacOS,999
2,HP 15S-EQ1040NS,"15.6"" HD",AMD 3020e,4GB RAM,128GB SSD,AMD Radeon™,229
3,HP Laptop 15s-fq3002ns,"15.6"" Full-HD",Intel® Celeron® N4500,8GB RAM,256GB SSD,Intel® UHD,529
4,ASUS Chromebook CX1400CNA-EK0179,"14"" Full-HD",Intel® Celeron® N3350,8 GB,64 GB,Chrome OS,999
5,Convertible 2 en 1 - Microsoft Surface Pro X,"13"" QHD+",Microsoft SQ® 1,8 GB RAM,128 GB SSD,Adreno 685,369
6,ASUS F1500EA-BQ2362,"15.6"" Full HD",Intel® Core™ i3-1115G4,8GB RAM,256 GB SSD,UHD Graphics,449
7,ASUS F1400EA-EK1838W,"14"" Full HD",Intel® Core™ i5-1135G7,8GB RAM,512GB SSD,Iris® Xe Graphics,599
8,Dynabook Satellite Pro C50-G-104,"15.6"" HD",Intel® Core™ i3-10110U,8GB RAM,256GB SSD,UHD,366
9,Apple MacBook Air (2020),"13.3"" Retina",Chip M1 de Apple,8 GB,256 GB SSD,MacOS,347


Exportamos el dataframe obtenido y ya estructurado para poder hacer analisis y distintos modelos de machine learning

In [233]:
MediaMarkt.to_csv('MediaMarkt.csv')

### Worten

Realizamos la misma limpieza y estructuracion de los datos para los de Worten

In [240]:
Worten

,0,1
0,Portátil HP 15s-fq5049ns (15.6'' - Intel Core ...,"749,00€"
1,Portátil ASUS F1500EA-EJ2382W (15.6 ''Intel Co...,"529,00€"
2,Portátil ACER Aspire 3 A315-58-56K2 (15.6'' - ...,"749,99€"
3,Portátil ACER Aspire 3 A315-58-79LA (15.6'' - ...,"699,00€"
4,Portátil HP 15s-eq2108ns (15.6'' - AMD Ryzen 5...,"599,99€"
5,Portátil HP 15S-FQ2168NS (15.6'' - Intel Core ...,"369,00€"
6,Portátil HP 15s-eq2111ns (15.6'' - AMD Ryzen 5...,"529,99€"
7,Portátil HP 15s-fq4000ns (15.6'' - Intel Core ...,"649,00€"
8,Portátil ASUS VivoBook 15 OLED K513EA-L12891W ...,"659,00€"
9,Macbook Air APPLE MGN63Y/A Gris Espacial (13.3...,"999,00€"


Limpiamos los nombres y separamos los datos para poder clasificarlos por sus caracteristicas

In [241]:
wortenName = [element.replace('Portátil ', '') for element in worten_laptop_name]
wortenName = [element.replace('Gaming', '') for element in wortenName]
wortenName = [element.replace(')', '') for element in wortenName]
wortenName = [element.replace('(', '') for element in wortenName]


wortenName = [x.split(' - ') for x in wortenName]

wortenName = pd.DataFrame(wortenName)
print(wortenName)

                                                    0  \
0                              HP 15s-fq5049ns 15.6''   
1    ASUS F1500EA-EJ2382W 15.6 ''Intel Core i5-1135G7   
2                   ACER Aspire 3 A315-58-56K2 15.6''   
3                   ACER Aspire 3 A315-58-79LA 15.6''   
4                              HP 15s-eq2108ns 15.6''   
5                              HP 15S-FQ2168NS 15.6''   
6                              HP 15s-eq2111ns 15.6''   
7                              HP 15s-fq4000ns 15.6''   
8         ASUS VivoBook 15 OLED K513EA-L12891W 15.6''   
9     Macbook Air APPLE MGN63Y/A Gris Espacial 13.3''   
10        MacBook Air 2020 APPLE Gris Espacial 13.3''   
11               MacBook Air 2020 APPLE Gris Espacial   
12                    Macbook Air APPLE Gris Espacial   
13   ASUS TUF Dash F15 FX516PM-HN023 Intel Core i7...   
14                    HP Chromebook 14a-na0013ns 14''   
15                      MacBook Air 2020 APPLE Dorado   
16                             

Luego eliminamos los datos sucios que se encuentran en otro formato diferente a los demas

In [242]:
wortenName = wortenName.drop([1])
wortenName = wortenName.drop([11])
wortenName = wortenName.drop([12])
wortenName = wortenName.drop([14])
wortenName = wortenName.drop([15])
wortenName = wortenName.drop([18])
wortenName = wortenName.drop([19])
wortenName = wortenName.drop([20])
wortenName = wortenName.drop([22])

del wortenName[5]
wortenName


,0,1,2,3,4
0,HP 15s-fq5049ns 15.6'',Intel Core i7-1255U,RAM: 16 GB,512 GB SSD,Intel Iris Xe Graphics
2,ACER Aspire 3 A315-58-56K2 15.6'',Intel Core i5-1135G7,RAM: 12 GB,512 GB SSD,Intel Iris Xe Graphics
3,ACER Aspire 3 A315-58-79LA 15.6'',Intel Core i7-1165G7,RAM: 16 GB,512 GB SSD,Intel Iris Xe Graphics
4,HP 15s-eq2108ns 15.6'',AMD Ryzen 5 5500U,RAM: 8 GB,512 GB SSD PCIe,AMD Radeon Graphics
5,HP 15S-FQ2168NS 15.6'',Intel Core i3-1115G4,RAM: 8 GB,256 GB SSD,Intel UHD Graphics
6,HP 15s-eq2111ns 15.6'',AMD Ryzen 5 5500U,RAM: 8 GB,512 GB SSD PCIe,AMD Radeon Graphics
7,HP 15s-fq4000ns 15.6'',Intel Core i5-1155G7,RAM: 8 GB,512 GB SSD,Intel Iris Xe Graphics
8,ASUS VivoBook 15 OLED K513EA-L12891W 15.6'',Intel Core i5-1135G7,RAM: 12 GB,512 GB SSD,Intel Iris Xe Graphics
9,Macbook Air APPLE MGN63Y/A Gris Espacial 13.3'',Apple M1,RAM: 8 GB,256 GB SSD,Integrada
10,MacBook Air 2020 APPLE Gris Espacial 13.3'',Apple M1,RAM: 16 GB,256 GB SSD,Integrada


Ahora limpiamos los datos del precio y eliminamos las que no se utilizaron como en el caso anterior

In [243]:
wortenPrice = [element.replace('€', '') for element in worten_laptop_price]

wortenPrice = pd.DataFrame(wortenPrice)
wortenPrice

,0
0,"749,00"
1,"529,00"
2,"749,99"
3,"699,00"
4,"599,99"
5,"369,00"
6,"529,99"
7,"649,00"
8,"659,00"
9,"999,00"


In [244]:
wortenPrice = wortenPrice.drop([1])
wortenPrice = wortenPrice.drop([11])
wortenPrice = wortenPrice.drop([12])
wortenPrice = wortenPrice.drop([14])
wortenPrice = wortenPrice.drop([15])
wortenPrice = wortenPrice.drop([18])
wortenPrice = wortenPrice.drop([19])
wortenPrice = wortenPrice.drop([20])
wortenPrice = wortenPrice.drop([22])
wortenPrice

,0
0,"749,00"
2,"749,99"
3,"699,00"
4,"599,99"
5,"369,00"
6,"529,99"
7,"649,00"
8,"659,00"
9,"999,00"
10,"1279,00"


Agrupamos ambos datos

In [245]:
Worten = pd.merge(wortenName, wortenPrice, left_index=True, right_index=True)
Worten

,0_x,1,2,3,4,0_y
0,HP 15s-fq5049ns 15.6'',Intel Core i7-1255U,RAM: 16 GB,512 GB SSD,Intel Iris Xe Graphics,"749,00"
2,ACER Aspire 3 A315-58-56K2 15.6'',Intel Core i5-1135G7,RAM: 12 GB,512 GB SSD,Intel Iris Xe Graphics,"749,99"
3,ACER Aspire 3 A315-58-79LA 15.6'',Intel Core i7-1165G7,RAM: 16 GB,512 GB SSD,Intel Iris Xe Graphics,"699,00"
4,HP 15s-eq2108ns 15.6'',AMD Ryzen 5 5500U,RAM: 8 GB,512 GB SSD PCIe,AMD Radeon Graphics,"599,99"
5,HP 15S-FQ2168NS 15.6'',Intel Core i3-1115G4,RAM: 8 GB,256 GB SSD,Intel UHD Graphics,"369,00"
6,HP 15s-eq2111ns 15.6'',AMD Ryzen 5 5500U,RAM: 8 GB,512 GB SSD PCIe,AMD Radeon Graphics,"529,99"
7,HP 15s-fq4000ns 15.6'',Intel Core i5-1155G7,RAM: 8 GB,512 GB SSD,Intel Iris Xe Graphics,"649,00"
8,ASUS VivoBook 15 OLED K513EA-L12891W 15.6'',Intel Core i5-1135G7,RAM: 12 GB,512 GB SSD,Intel Iris Xe Graphics,"659,00"
9,Macbook Air APPLE MGN63Y/A Gris Espacial 13.3'',Apple M1,RAM: 8 GB,256 GB SSD,Integrada,"999,00"
10,MacBook Air 2020 APPLE Gris Espacial 13.3'',Apple M1,RAM: 16 GB,256 GB SSD,Integrada,"1279,00"


In [247]:
Worten.rename(columns = {'0_x':'Nombre Ordenador/Screen', 1:'Processor', 2: 'RAM Memory', 3: 'Hard Drive', 4: 'Graphics Card', '0_y': 'Price (€)' }, inplace = True)
Worten

,Nombre Ordenador/Screen,Processor,RAM Memory,Hard Drive,Graphics Card,Price (€)
0,HP 15s-fq5049ns 15.6'',Intel Core i7-1255U,RAM: 16 GB,512 GB SSD,Intel Iris Xe Graphics,"749,00"
2,ACER Aspire 3 A315-58-56K2 15.6'',Intel Core i5-1135G7,RAM: 12 GB,512 GB SSD,Intel Iris Xe Graphics,"749,99"
3,ACER Aspire 3 A315-58-79LA 15.6'',Intel Core i7-1165G7,RAM: 16 GB,512 GB SSD,Intel Iris Xe Graphics,"699,00"
4,HP 15s-eq2108ns 15.6'',AMD Ryzen 5 5500U,RAM: 8 GB,512 GB SSD PCIe,AMD Radeon Graphics,"599,99"
5,HP 15S-FQ2168NS 15.6'',Intel Core i3-1115G4,RAM: 8 GB,256 GB SSD,Intel UHD Graphics,"369,00"
6,HP 15s-eq2111ns 15.6'',AMD Ryzen 5 5500U,RAM: 8 GB,512 GB SSD PCIe,AMD Radeon Graphics,"529,99"
7,HP 15s-fq4000ns 15.6'',Intel Core i5-1155G7,RAM: 8 GB,512 GB SSD,Intel Iris Xe Graphics,"649,00"
8,ASUS VivoBook 15 OLED K513EA-L12891W 15.6'',Intel Core i5-1135G7,RAM: 12 GB,512 GB SSD,Intel Iris Xe Graphics,"659,00"
9,Macbook Air APPLE MGN63Y/A Gris Espacial 13.3'',Apple M1,RAM: 8 GB,256 GB SSD,Integrada,"999,00"
10,MacBook Air 2020 APPLE Gris Espacial 13.3'',Apple M1,RAM: 16 GB,256 GB SSD,Integrada,"1279,00"


Exportamos el dataframe de Worten

In [249]:
Worten.to_csv('Worten.csv')

### PhoneHouse

In [161]:
PhoneHouse

,0,1
0,Samsung Galaxy Book2 Pro 15 i7 512GB+16GB RAM,1549€
1,Samsung Galaxy Book2 15 INT i5 512GB+8GB RAM,849€
2,Lenovo ThinkPad Yoga 370 Negro Híbrido (2-en-1...,379€
3,HP ZBook Estación de trabajo móvil Studio G4,789€
4,HP ZBook Studio Estación de trabajo portátil G3,639€
5,Toshiba Portégé Z30t-C-133,1995€
6,Toshiba Tecra X40-D-10G,"621,48€"
7,Microsoft Surface Laptop CI5 256GB 8GB SYST 13...,"2018,43€"
8,"Lenovo IdeaPad Y700 2.6GHz I7-6700HQ 15.6"""" Negro","1133,21€"
9,HP EliteBook Ordenador portátil 1040 G4,445€


Al visualizar los datos nos damos cuenta de que no son posibles de trabajar ni limpiar en conjunto debido a que no tienen una estructura establecida y no es posible agruparla al dataframe que se va a utilizar para el MachineLearning <br>
De igual manera se limpia lo mas que se pueda para poder visualizar los nombres, caracteristicas y precios de una manera mas facil

In [258]:
phonehouseName = [element.replace('\r\n\t\t\t\t\t', '') for element in ph_laptop_name]
phonehouseName = [element.replace('\r\n\t\t\t\t', '') for element in phonehouseName]

phonehouseName = pd.DataFrame(phonehouseName)
print(phonehouseName)

                                                    0
0       Samsung Galaxy Book2 Pro 15 i7 512GB+16GB RAM
1        Samsung Galaxy Book2 15 INT i5 512GB+8GB RAM
2   Lenovo ThinkPad Yoga 370 Negro Híbrido (2-en-1...
3                          Toshiba Portégé Z30t-C-133
4                             Toshiba Tecra X40-D-10G
5   Microsoft Surface Laptop CI5 256GB 8GB SYST 13...
6   Lenovo IdeaPad Y700 2.6GHz I7-6700HQ 15.6"" Negro
7   Lenovo ThinkPad E580 Negro Portátil 39,6 cm (1...
8   Apple MacBook Air Plata Portátil 29,5 cm (11.6...
9   Lenovo ThinkPad X280 Negro Portátil 31,8 cm (1...
10  Lenovo ThinkPad L470 Negro Portátil 35,6 cm (1...
11  Apple MacBook Air 13.3 (1.8GHz/8GB RAM/128GB SSD)
12  Apple MacBook Pro Gris Portátil 39,1 cm (15.4)...
13  Apple MacBook Pro Gris Portátil 33,8 cm (13.3)...
14  Apple MacBook Pro Plata Portátil 33,8 cm (13.3...
15  Apple MacBook Pro Plata Portátil 33,8 cm (13.3...
16  Medion Portatil  Akoya P6687 Md60913 15.6/I5-8...
17  MSI PORTATIL MSI WS63 8S

Limpiamos los precios de los portatiles

In [256]:
PhoneHousePrice = [element.replace('€', '') for element in ph_laptop_price]

PhoneHousePrice = pd.DataFrame(PhoneHousePrice)
PhoneHousePrice

,0
0,1549
1,849
2,379
3,1995
4,"621,48"
5,"2018,43"
6,"1133,21"
7,479
8,310
9,"369,95"


Agrupamos los nombres y sus caracteristicas (sin estructura) junto con los precios ya limpios

In [259]:
PhoneHouse = pd.merge(phonehouseName, PhoneHousePrice, left_index=True, right_index=True)
PhoneHouse

,0_x,0_y
0,Samsung Galaxy Book2 Pro 15 i7 512GB+16GB RAM,1549
1,Samsung Galaxy Book2 15 INT i5 512GB+8GB RAM,849
2,Lenovo ThinkPad Yoga 370 Negro Híbrido (2-en-1...,379
3,Toshiba Portégé Z30t-C-133,1995
4,Toshiba Tecra X40-D-10G,"621,48"
5,Microsoft Surface Laptop CI5 256GB 8GB SYST 13...,"2018,43"
6,"Lenovo IdeaPad Y700 2.6GHz I7-6700HQ 15.6"""" Negro","1133,21"
7,"Lenovo ThinkPad E580 Negro Portátil 39,6 cm (1...",479
8,"Apple MacBook Air Plata Portátil 29,5 cm (11.6...",310
9,"Lenovo ThinkPad X280 Negro Portátil 31,8 cm (1...","369,95"


In [260]:
PhoneHouse.rename(columns = {'0_x':'Nombre Ordenador/Caracteristicas','0_y': 'Price (€)' }, inplace = True)
PhoneHouse

,Nombre Ordenador/Caracteristicas,Price (€)
0,Samsung Galaxy Book2 Pro 15 i7 512GB+16GB RAM,1549
1,Samsung Galaxy Book2 15 INT i5 512GB+8GB RAM,849
2,Lenovo ThinkPad Yoga 370 Negro Híbrido (2-en-1...,379
3,Toshiba Portégé Z30t-C-133,1995
4,Toshiba Tecra X40-D-10G,"621,48"
5,Microsoft Surface Laptop CI5 256GB 8GB SYST 13...,"2018,43"
6,"Lenovo IdeaPad Y700 2.6GHz I7-6700HQ 15.6"""" Negro","1133,21"
7,"Lenovo ThinkPad E580 Negro Portátil 39,6 cm (1...",479
8,"Apple MacBook Air Plata Portátil 29,5 cm (11.6...",310
9,"Lenovo ThinkPad X280 Negro Portátil 31,8 cm (1...","369,95"


Exportamos el dataframe

In [261]:
PhoneHouse.to_csv('PhoneHouse.csv')

### BackMarket

In [262]:
BackMarket

,0,1
0,\n HP EliteBook 840 G3 14” (2016)\n...,"\n 254,30 €\n"
1,\n HP EliteBook 840 G3 14” (2020)\n...,"\n 399,00 €\n"
2,\n Lenovo ThinkPad T470 14” (2017)\...,"\n 292,92 €\n"
3,\n Lenovo ThinkPad T470 14” (2017)\...,"\n 349,00 €\n"
4,\n HP EliteBook 840 G3 14” (2020)\n...,"\n 344,00 €\n"
5,\n Dell Latitude E5450 14” (2015)\n...,"\n 199,00 €\n"
6,\n HP EliteBook 840 G3 14” (2020)\n...,"\n 459,00 €\n"
7,"\n HP ProBook x360 11 G3 11,6” (Mar...","\n 199,00 €\n"
8,\n HP EliteBook 840 G3 14” (2020)\n...,"\n 450,00 €\n"
9,\n HP EliteBook 840 G3 14” (2016)\n...,"\n 258,36 €\n"


Facilmente se puede notar que estos datos necesitan mucha limpieza para poder visualizarlos de una mejor manera y para poder trabajarlos

In [263]:
backmarketName = [element.replace('\n','') for element in BackMarket[0]]

backmarketName = pd.DataFrame(backmarketName)
print(backmarketName)

                                                    0
0               HP EliteBook 840 G3 14” (2016)    ...
1               HP EliteBook 840 G3 14” (2020)    ...
2               Lenovo ThinkPad T470 14” (2017)   ...
3               Lenovo ThinkPad T470 14” (2017)   ...
4               HP EliteBook 840 G3 14” (2020)    ...
5               Dell Latitude E5450 14” (2015)    ...
6               HP EliteBook 840 G3 14” (2020)    ...
7               HP ProBook x360 11 G3 11,6” (Marzo...
8               HP EliteBook 840 G3 14” (2020)    ...
9               HP EliteBook 840 G3 14” (2016)    ...
10              Lenovo Thinkpad Yoga 260 12,5” (20...
11              Dell Latitude 3300 13,3” (2019)   ...
12              HP EliteBook 840 G5 14” (2021)    ...
13              Toshiba Satellite Pro C50-E-103 15...
14              HP EliteBook 840 G2 14” (2015)    ...
15              Lenovo ThinkPad T450 14” (2015)   ...
16              Lenovo Thinkpad T460 14” (2016)   ...
17              HP EliteBook

In [264]:
backmarketPrice = [element.replace('€', '') for element in bm_laptop_price]
backmarketPrice = [element.replace('\n','') for element in backmarketPrice]

backmarketPrice = pd.DataFrame(backmarketPrice)
backmarketPrice

,0
0,"254,30"
1,"399,00"
2,"292,92"
3,"349,00"
4,"344,00"
5,"199,00"
6,"459,00"
7,"199,00"
8,"450,00"
9,"258,36"


Agrupamos los datos en un mismo dataframe

In [265]:
BackMarket = pd.merge(backmarketName, backmarketPrice, left_index=True, right_index=True)
BackMarket

,0_x,0_y
0,HP EliteBook 840 G3 14” (2016) ...,"254,30"
1,HP EliteBook 840 G3 14” (2020) ...,"399,00"
2,Lenovo ThinkPad T470 14” (2017) ...,"292,92"
3,Lenovo ThinkPad T470 14” (2017) ...,"349,00"
4,HP EliteBook 840 G3 14” (2020) ...,"344,00"
5,Dell Latitude E5450 14” (2015) ...,"199,00"
6,HP EliteBook 840 G3 14” (2020) ...,"459,00"
7,"HP ProBook x360 11 G3 11,6” (Marzo...","199,00"
8,HP EliteBook 840 G3 14” (2020) ...,"450,00"
9,HP EliteBook 840 G3 14” (2016) ...,"258,36"


In [266]:
BackMarket.rename(columns = {'0_x':'Nombre Ordenador / Caracteristicas', '0_y': 'Price (€)' }, inplace = True)
BackMarket

,Nombre Ordenador / Caracteristicas,Price (€)
0,HP EliteBook 840 G3 14” (2016) ...,"254,30"
1,HP EliteBook 840 G3 14” (2020) ...,"399,00"
2,Lenovo ThinkPad T470 14” (2017) ...,"292,92"
3,Lenovo ThinkPad T470 14” (2017) ...,"349,00"
4,HP EliteBook 840 G3 14” (2020) ...,"344,00"
5,Dell Latitude E5450 14” (2015) ...,"199,00"
6,HP EliteBook 840 G3 14” (2020) ...,"459,00"
7,"HP ProBook x360 11 G3 11,6” (Marzo...","199,00"
8,HP EliteBook 840 G3 14” (2020) ...,"450,00"
9,HP EliteBook 840 G3 14” (2016) ...,"258,36"


Exportamos los datos

In [267]:
BackMarket.to_csv('BackMarket.csv')